In [1]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
 
df = pd.read_csv('face_data.csv')
n_persons = df['target'].nunique() 
X = np.array(df.drop('target', axis=1)) # 400 x 4096
y = np.array(df['target'])
 
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) # deafult test_size=0.25
 
# prepare data for PyTorch Tensor
X_train = torch.from_numpy(X_train).float() # convert to float tensor
y_train = torch.from_numpy(y_train).float() # 
train_dataset = TensorDataset(X_train, y_train) # create your datset
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()
test_dataset = TensorDataset(X_test, y_test) # create your datset
 
# create dataloader for PyTorch
batch_size = 64 # 32, 64, 128, 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # convert to dataloader
test_loader = DataLoader(test_dataset, batch_size=len(X_test), shuffle=False)

In [2]:
import torch.nn as nn
import torch.nn.functional as F
 
# select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = "cpu" # run faster than cuda in some cases
print("Using {} device".format(device))
 
# Create a neural network
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(64*64, 512), # image length 64x64=4096,  fully connected layer
            nn.ReLU(), # try to take ReLU out to see what happen
            nn.Linear(512, 128), # second hidden layer
            nn.ReLU(),
            nn.Linear(128, 40) # 40 classes,  fully connected layer
            # nn.Softmax()
        )
    # Specify how data will pass through this model
    def forward(self, x):
        # out = self.mlp(x) 
 
        # Apply softmax to x here~
        x = self.mlp(x)
        out = F.log_softmax(x, dim=1) # it’s faster and has better numerical propertie than softmax
        # out = F.softmax(x, dim=1)
        return out
 
 
# define model, optimizer, loss function
model = MLP().to(device) # start an instance
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # default lreaning rate=1e-3
loss_fun = nn.CrossEntropyLoss() # define loss function
 
print(model)


Using cpu device
MLP(
  (mlp): Sequential(
    (0): Linear(in_features=4096, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=40, bias=True)
  )
)


In [3]:
input = torch.randn(32, 64 * 64) 
m1 = nn.Linear(64*64, 512)
output = m1(input)
print(output.size())
output = F.relu(output)
print(output.size())
m2 = nn.Linear(512, 40)
output = m2(output)
print(output.size())
output = F.log_softmax(output, dim=1)
print(output.size())

torch.Size([32, 512])
torch.Size([32, 512])
torch.Size([32, 40])
torch.Size([32, 40])


#### start trining

In [5]:
from tqdm import tqdm
 
epochs = 50 # Repeat the whole dataset epochs times
model.train() # Sets the module in training mode. The training model allow the parameters to be updated during backpropagation.
for epoch in range(epochs):
# for epoch in tqdm(range(epochs)):
    trainAcc = 0
    samples = 0
    losses = []
    for batch_num, input_data in enumerate(train_loader):
    # for batch_num, input_data in tqdm(enumerate(train_loader), total=len(train_loader)):
         
        x, y = input_data
        x = x.to(device).float()
        y = y.to(device)
 
        # perform training based on the backpropagation
        y_pre = model(x) # predict y
        loss = loss_fun(y_pre, y.long()) # the loss function nn.CrossEntropyLoss()
        losses.append(loss.item())
 
        optimizer.zero_grad() # Zeros the gradients accumulated from the previous batch/step of the model
        loss.backward() # Performs backpropagation and calculates the gradient for each parameter
        optimizer.step() # updates the weights based on the gradients of the parameters.
         
        # Record the training accuracy for each batch
        trainAcc += (y_pre.argmax(dim=1) == y).sum().item() # comparison
        samples += y.size(0)
        if batch_num % 4 == 0:
            print('\tEpoch %d | Batch %d | Loss %6.2f' % (epoch, batch_num, loss.item()))
    print('Epoch %d | Loss %6.2f | train accuracy %.4f' % (epoch, sum(losses)/len(losses), trainAcc/samples))
 
print('Finished ... Loss %7.4f | train accuracy %.4f' % (sum(losses)/len(losses), trainAcc/samples))


	Epoch 0 | Batch 0 | Loss   3.71
	Epoch 0 | Batch 4 | Loss   3.75
Epoch 0 | Loss   3.77 | train accuracy 0.0214
	Epoch 1 | Batch 0 | Loss   3.79
	Epoch 1 | Batch 4 | Loss   3.74
Epoch 1 | Loss   3.73 | train accuracy 0.0321
	Epoch 2 | Batch 0 | Loss   3.67
	Epoch 2 | Batch 4 | Loss   3.66
Epoch 2 | Loss   3.64 | train accuracy 0.0321
	Epoch 3 | Batch 0 | Loss   3.61
	Epoch 3 | Batch 4 | Loss   3.74
Epoch 3 | Loss   3.64 | train accuracy 0.0536
	Epoch 4 | Batch 0 | Loss   3.59
	Epoch 4 | Batch 4 | Loss   3.60
Epoch 4 | Loss   3.59 | train accuracy 0.0536
	Epoch 5 | Batch 0 | Loss   3.54
	Epoch 5 | Batch 4 | Loss   3.66
Epoch 5 | Loss   3.58 | train accuracy 0.0607
	Epoch 6 | Batch 0 | Loss   3.45
	Epoch 6 | Batch 4 | Loss   3.67
Epoch 6 | Loss   3.56 | train accuracy 0.0679
	Epoch 7 | Batch 0 | Loss   3.47
	Epoch 7 | Batch 4 | Loss   3.48
Epoch 7 | Loss   3.48 | train accuracy 0.0929
	Epoch 8 | Batch 0 | Loss   3.42
	Epoch 8 | Batch 4 | Loss   3.33
Epoch 8 | Loss   3.43 | train accuracy

In [6]:
model.eval() 
testAcc = 0
samples = 0
with torch.no_grad():
    for x, y_truth in test_loader:
        x = x.to(device).float()
        y_truth = y_truth.to(device)
        y_pre = model(x).argmax(dim=1) # the predictions for the batch
        testAcc += (y_pre == y_truth).sum().item() # comparison
        samples += y_truth.size(0)
 
    print('Test Accuracy:{:.3f}'.format(testAcc/samples))

Test Accuracy:0.767


In [7]:
import csv
 
# use eval() in conjunction with a torch.no_grad() context, 
# meaning that gradient computation is turned off in evaluation mode
model.eval() 
testAcc = 0
samples = 0
 
with open('mlp_att.csv', 'w') as f:
    fieldnames = ['ImageId', 'Label', 'Ground_Truth']
    writer = csv.DictWriter(f, fieldnames=fieldnames, lineterminator = '\n')
    writer.writeheader()
    image_id = 1
 
    with torch.no_grad():
        for x, y_truth in test_loader:
            x = x.to(device).float()
            y_truth = y_truth.to(device).long()
            yIdx = 0
            y_pre = model(x).argmax(dim=1) # the predictions for the batch
            testAcc += (y_pre == y_truth).sum().item() # comparison
            samples += y_truth.size(0)
            for y in y_pre:
                writer.writerow({fieldnames[0]: image_id,fieldnames[1]: y.item(), fieldnames[2]: y_truth[yIdx].item()})
                image_id += 1
                yIdx += 1
 
        print('Test Accuracy:{:.3f}'.format(testAcc/samples))

Test Accuracy:0.767


### 範例 2：PyTorch 的深度機器學習 CNN 範例：以手寫數字辨識為例。程式部分參考官網範例：

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import pickle
import os
import numpy as np

